<a href="https://colab.research.google.com/github/vit050587/Natural_Language_Processing/blob/master/KVA_HW_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Урок 13. Модель BERT и GPT

Взять датасет

https://huggingface.co/datasets/merionum/ru_paraphraser

решить задачу парафраза

In [1]:
!pip install transformers sentencepiece --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.8 MB/s eta 0:00:00


In [2]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'paraphrases'],
        num_rows: 1725393
    })
})

In [5]:
corpus['train'][0:4]

{'id': ['0', '1', '2', '3'],
 'paraphrases': [['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
   'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем'],
  ['В Индии 5 млн женщин выстроились стеной в 620 км ради равенства (Фото)',
   'Индийские женщины выстроили живую стену длиной 620 километров, требуя равенства',
   'В Индии женщины встали живой стеной длиной 620 км'],
  ['Томас Бах: "Олимпийский комитет России понес достаточное наказание"',
   'Президент МОК Бах: Россия уже достаточно наказана за допинг',
   'Глава МОК заявил, что ОКР понес наказание за манипуляции с пробами на ОИ в Сочи'],
  ['Россия не представила WADA данные по пробам в срок и может быть наказана',
   'WADA: Россия не предоставила нам доступ к допинг-пробам',
   'В WADA пожаловались на отсутствие доступа к допинг-пробам российских спортсменов']]}

#Используем предобученную модель 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

In [6]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

In [7]:
def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()

    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    if p[1] > 0.5:
        paraphrase = "paraphrase"
    else:
        paraphrase = "non"

    return [text1, text2, p[0], p[1], paraphrase]

In [8]:
print(compare_texts('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))
# [0.7056226 0.2943774]
print(compare_texts('Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась'))

['Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода', 0.70562273, 0.29437727, 'non']
['Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась', 0.16524385, 0.83475614, 'paraphrase']


#Используем предобученную модель 'cointegrated/rut5-base-paraphraser'

In [9]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name_gen = 'cointegrated/rut5-base-paraphraser'
model_gen = T5ForConditionalGeneration.from_pretrained(model_name_gen)
tokenizer = T5Tokenizer.from_pretrained(model_name_gen)
model_gen.cuda();
model_gen.eval();

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [10]:
def get_paraphrase(text, beams=5, grams=4, do_sample=False):

    x = tokenizer(text, return_tensors='pt', padding=True).to(model_gen.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model_gen.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)

    return tokenizer.decode(out[0], skip_special_tokens=True)

In [11]:
print(get_paraphrase('Сегодня на улице хорошая погода.'))

Сегодня погода на улице очень хорошая.
